In [1]:
import requests
from bs4 import BeautifulSoup

# Function to get html soup
def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None
        

In [11]:
urls = []
urls.append(input("please enter a comma separated list of websites:"))


please enter websites:http://one.com, http://two.com
['http://one.com, http://two.com']


In [2]:
# List of urls to crawl

#urls = ['https://thedatadean.co.uk']

# Initialize dictionary and list
soups = {}
crawled_urls = []

#Fetch soup for each url in list and add to crawled urls
def CrawlPage(urls):
    for url in urls:
        soup = fetch_page(url)
        soups[url] = soup
        crawled_urls.append(url)
        
# Crawl user provided list of urls
CrawlPage(urls)
print(soups)

# Empty urls list
urls = []

print(crawled_urls)
print(urls)

{'https://thedatadean.co.uk': 
<!DOCTYPE html>
 <!--[if lt IE 7]> <html class="no-js ie lt-ie10 lt-ie9 lt-ie8 lt-ie7"> <![endif]--> <!--[if IE 7]> <html class="no-js ie ie7 lt-ie10 lt-ie9 lt-ie8"> <![endif]--> <!--[if IE 8]> <html class="no-js ie ie8 lt-ie10 lt-ie9"> <![endif]--> <!--[if IE 9]> <html class="no-js ie ie9 lt-ie10"> <![endif]--> <!--[if gt IE 9]> <html class="no-js ie not-ie-legacy"> <![endif]--> <!--[if !IE]><!--> <html class="wdk-theme no-js font-size--med"> <!--<![endif]--> <head> <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1.0" name="viewport"/> <meta content="telephone=no" name="format-detection"/> <link href="https://assetsbeta.create-cdn.net/_assets/livesites/feature/content-builder/content-builder.min.css?7d12e0ce340629966bbb783ae9891852" rel="stylesheet"/> <title>Dean Walsh's Portfolio Site</title> <meta content="" name="keywords"/> <meta content="Hire an entry level data analyst with skills in 

In [3]:
# Scrape links into list
internal_links = []
external_links = []
links = []

# For each entry in soups 
for url, soup in soups.items():

# Find all links in soup

    for link in soup.find_all("a"):
        links.append(link.get('href'))

# Remove 'None' entries from list
    new_links = [item for item in links if item is not None]

# Get full link for interal links and separate internal and external links
# WILL FAIL IF URL IS NOT ROOT
    i = 0
    while i < len(new_links):
        if new_links[i].startswith('http') and new_links[i].startswith(url):
            internal_links.append(new_links[i])
        elif new_links[i].startswith('http'):
            external_links.append(new_links[i])
        else:
            internal_links.append(url + new_links[i]) 
        i += 1

# Convert to set
internal_links = set(internal_links)
external_links = set(external_links)

# Add internal links to urls and crawl
for link in internal_links:
    urls.append(link)

#CrawlPage(urls)

# ! Only https links are recognized as starting with url when original url is https, http links are being put into external links.

In [4]:
# create index of words

import re
indices = {}

def index_words(soup):
    index = {}
# Get text and title
    words = re.findall(r'\w+', soup.get_text())
    title = soup.find_all("title")
# Sometimes no title is found; use empty string in these cases
    if title:
        title = title[0].text
    else:
        title = ""
        
    for word in words:
        word = word.lower()
# Create word counts and give double weight if word is in the title
        if word in index and word in title.lower():
            index[word] += 2
        elif word in index and word not in title.lower():
             index[word] += 1
        elif word not in index and word in title.lower():
            index[word] = 2
        else:
            index[word] = 1
    return index

# create new dictionary with indices


indices = {key: index_words(value) for key, value in soups.items()}

    

In [5]:
#remove stop words
#will probably need my own custom list with additions eventually
#add video, fan, fans, comments, shares, hours, hour, minute, minutes, story, stories, exclusive,world, news, latest, updates, report, thing, daily, issues, breitbart,zerohedge,express

stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = set(stopwords_list.decode().splitlines()) 

def remove_stop_words(index):
    for stop_word in stopwords:
        if stop_word in index:
            del index[stop_word]
    return index


for index in indices.values():
    remove_stop_words(index)


print(indices)
#print(len(indices['https://www.zerohedge.com/']))

{'https://thedatadean.co.uk': {'dean': 10, 'walsh': 4, 'portfolio': 8, 'site': 2, 'data': 5, 'contact': 1, 'introducing': 1, 'scott': 1, 'hire': 1, 'download': 1, 'employment': 1, 'status': 1, 'certified': 1, 'excel': 3, 'expert': 1, 'postgresql': 1, 'sql': 1, 'server': 1, 'cloud': 1, 'platforms': 1, 'gcp': 1, 'azure': 1, 'tableau': 1, 'power': 1, 'basic': 1, 'python': 2, 'programming': 1, 'projectsthe': 1, 'projects': 2, 'created': 1, 'purely': 1, 'based': 1, 'fabricated': 1, 'sources': 1, 'report': 1, 'exploratory': 1, 'analysis': 1, 'dashboard': 1, 'view': 1, 'homeaboutportfoliocontact': 1, 'copyright': 1, '2024': 1, 'rights': 1, 'reserved': 1, 'designed': 1, 'create': 1}, 'https://www.zerohedge.com/': {'zerohedge': 12, 'homepremiumcontributorschannelsallpartnersthe': 1, 'market': 1, 'earspotgammadebatesaboutmorerssdonateadvertisemerchformer': 1, 'cdc': 1, 'director': 1, 'bird': 1, 'flu': 1, 'highly': 1, 'infectious': 1, 'humans': 1, 'months': 1, 'tdbcovid': 1, 'propaganda': 1, 'rou

In [6]:
#define top topics for a page

def PageTopics(dictionary):
    # Filter dictionary for keys with values greater than one
    filtered_dict = {k: v for k, v in dictionary.items() if v > 1}
    # Sort keys by value in descending order
    sorted_keys = sorted(filtered_dict, key=filtered_dict.get, reverse=True)
    sorted_dict = {key: filtered_dict[key] for key in sorted_keys}
    return sorted_dict

#create new list of only top topics

top_index = {key: PageTopics(value) for key, value in indices.items()}



print(top_index)


{'https://thedatadean.co.uk': {'dean': 10, 'portfolio': 8, 'data': 5, 'walsh': 4, 'excel': 3, 'site': 2, 'python': 2, 'projects': 2}, 'https://www.zerohedge.com/': {'zerohedge': 12, 'oil': 5, 'read': 3, 'gold': 3, '0the': 3, 'policy': 3, 'year': 2, 'years': 2, 'pain': 2, 'trades': 2, 'bragg': 2, 'office': 2, 'deleted': 2, 'records': 2, 'attorney': 2, '0goldman': 2, 'biden': 2, '2024': 2, 'consumer': 2, 'activists': 2, 'destroy': 2, 'case': 2, 'investment': 2, 'drone': 2, 'russia': 2, 'irs': 2, 'tax': 2, 'exempt': 2, 'status': 2, 'war': 2, 'fresh': 2, 'push': 2, 'kharkov': 2, 'click': 2, 'hedge': 2, 'discrimination': 2}}


In [ ]:
#probably need to apply stemming at some point


In [7]:
# Merge indices from every url to get a single index for this run

run_index = {}
for inner_dict in top_index.values():
    for key, value in inner_dict.items():
        run_index[key] = run_index.get(key, 0) + value


final_run_index = PageTopics(run_index)

print(final_run_index)

{'zerohedge': 12, 'dean': 10, 'portfolio': 8, 'data': 5, 'oil': 5, 'walsh': 4, 'excel': 3, 'read': 3, 'gold': 3, '0the': 3, 'policy': 3, 'site': 2, 'python': 2, 'projects': 2, 'year': 2, 'years': 2, 'pain': 2, 'trades': 2, 'bragg': 2, 'office': 2, 'deleted': 2, 'records': 2, 'attorney': 2, '0goldman': 2, 'biden': 2, '2024': 2, 'consumer': 2, 'activists': 2, 'destroy': 2, 'case': 2, 'investment': 2, 'drone': 2, 'russia': 2, 'irs': 2, 'tax': 2, 'exempt': 2, 'status': 2, 'war': 2, 'fresh': 2, 'push': 2, 'kharkov': 2, 'click': 2, 'hedge': 2, 'discrimination': 2}
